# udp_tcp_rate_analysis

In [9]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/iot/tuple/tuple*',sep='\t',header=True)
#d.show(3)

d = d.select(from_unixtime('time').alias('ts'),regexp_replace('mac', ':', '').alias('mac'),'bytes_out','pkts_out','proto','dip','dport').sort('ts',ascending=True).dropna().distinct()
d = d.withColumn('mac1_8',substring(d.mac, 1, 8))
#d.show(truncate=False)

p = spark.read.csv('/user/maxnet/iot/tuple/oui8.csv',header=True)
#p.show(3)

p_sxt = p.filter(p.类型 == '智能摄像头')
p_sxt = p_sxt.drop('term_id','中文名称')

d_sxt = d.join(p_sxt,d.mac1_8 == p_sxt.oui8,how = 'left').distinct().dropna()
#d_sxt.show(3)

ys = d_sxt.filter(d_sxt.brand == '萤石').select('ts','mac','bytes_out','pkts_out','proto','dip','dport').distinct()

In [2]:
ys.count(),ys.distinct().count()

(317506, 317506)

## 私网ip过滤

In [10]:
ys = ys.filter(~ys.dip.startswith('10.'))
ys.count()

317014

In [4]:
ys.filter(ys.dip.startswith('172')).show(truncate=False)

+-------------------+------------+---------+--------+-----+--------------+-----+
|ts                 |mac         |bytes_out|pkts_out|proto|dip           |dport|
+-------------------+------------+---------+--------+-----+--------------+-----+
|2019-04-06 21:34:10|BC1C8115ABCD|296      |4       |6    |172.45.185.206|33001|
|2019-04-07 21:17:13|4419B6A770D3|3760     |16      |17   |172.25.174.208|10101|
|2019-04-25 14:37:19|4419B6A770D3|3760     |16      |17   |172.23.218.66 |10101|
+-------------------+------------+---------+--------+-----+--------------+-----+



In [11]:
ys = ys.filter(~ys.dip.startswith('172.23.'))
#ys.count()

In [12]:
ys = ys.filter(~ys.dip.startswith('172.25.'))
#ys.count()

In [13]:
ys = ys.filter(~ys.dip.startswith('192.168.'))
#ys.count()

In [14]:
ys.select('proto').distinct().show()

+-----+
|proto|
+-----+
|   17|
|    6|
|    1|
+-----+



In [87]:
ys.select('ts','proto').distinct().count()

35706

In [88]:
df = ys.select('ts','proto','dip','dport').distinct() 

In [89]:
df.show(3)

+-------------------+-----+---------------+-----+
|                 ts|proto|            dip|dport|
+-------------------+-----+---------------+-----+
|2019-03-25 15:01:27|   17|183.136.184.150| 6003|
|2019-03-25 15:01:27|   17|183.136.184.149| 6002|
|2019-03-25 15:01:27|    6|183.136.184.149| 6002|
+-------------------+-----+---------------+-----+
only showing top 3 rows



In [90]:
df.groupBy('proto').count().show()

+-----+------+
|proto| count|
+-----+------+
|   17|232631|
|    6| 28165|
|    1|  4685|
+-----+------+



In [91]:
dd = df.dropna().sort('ts',ascending=True)

In [92]:
w1 = dd.groupBy(window('ts', '1 day'),'proto').count()
w1.sort('window',ascending=True).show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |proto|count|
+------------------------------------------+-----+-----+
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|17   |1908 |
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|1    |88   |
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|6    |730  |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|17   |4370 |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|6    |934  |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|1    |136  |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|6    |529  |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|17   |1219 |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|1    |100  |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|17   |401  |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|1    |36   |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|6    |392  |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|17   |8007 |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|1    |102  |
|[2019-03-29 08:00:00, 2019-03-

In [93]:
w1_tmp = w1.sort('window',ascending=True).filter(w1.proto != '1')
w1_tmp.show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |proto|count|
+------------------------------------------+-----+-----+
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|17   |1908 |
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|6    |730  |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|17   |4370 |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|6    |934  |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|17   |1219 |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|6    |529  |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|17   |401  |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|6    |392  |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|17   |8007 |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|6    |508  |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|6    |301  |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|17   |3574 |
|[2019-03-31 08:00:00, 2019-04-01 08:00:00]|6    |500  |
|[2019-03-31 08:00:00, 2019-04-01 08:00:00]|17   |3437 |
|[2019-04-01 08:00:00, 2019-04-

In [94]:
w1_udp = w1_tmp.sort('window',ascending=True).filter(w1_tmp.proto == '17')
w1_udp.show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |proto|count|
+------------------------------------------+-----+-----+
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|17   |1908 |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|17   |4370 |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|17   |1219 |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|17   |401  |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|17   |8007 |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|17   |3574 |
|[2019-03-31 08:00:00, 2019-04-01 08:00:00]|17   |3437 |
|[2019-04-01 08:00:00, 2019-04-02 08:00:00]|17   |372  |
|[2019-04-02 08:00:00, 2019-04-03 08:00:00]|17   |1034 |
|[2019-04-03 08:00:00, 2019-04-04 08:00:00]|17   |9864 |
|[2019-04-04 08:00:00, 2019-04-05 08:00:00]|17   |2710 |
|[2019-04-05 08:00:00, 2019-04-06 08:00:00]|17   |1643 |
|[2019-04-06 08:00:00, 2019-04-07 08:00:00]|17   |1442 |
|[2019-04-07 08:00:00, 2019-04-08 08:00:00]|17   |5185 |
|[2019-04-08 08:00:00, 2019-04-

In [95]:
w1_tcp = w1_tmp.sort('window',ascending=True).filter(w1_tmp.proto == '6')
w1_tcp.show(truncate=False)

+------------------------------------------+-----+-----+
|window                                    |proto|count|
+------------------------------------------+-----+-----+
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|6    |730  |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|6    |934  |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|6    |529  |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|6    |392  |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|6    |508  |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|6    |301  |
|[2019-03-31 08:00:00, 2019-04-01 08:00:00]|6    |500  |
|[2019-04-01 08:00:00, 2019-04-02 08:00:00]|6    |197  |
|[2019-04-02 08:00:00, 2019-04-03 08:00:00]|6    |630  |
|[2019-04-03 08:00:00, 2019-04-04 08:00:00]|6    |632  |
|[2019-04-04 08:00:00, 2019-04-05 08:00:00]|6    |634  |
|[2019-04-05 08:00:00, 2019-04-06 08:00:00]|6    |646  |
|[2019-04-06 08:00:00, 2019-04-07 08:00:00]|6    |688  |
|[2019-04-07 08:00:00, 2019-04-08 08:00:00]|6    |505  |
|[2019-04-08 08:00:00, 2019-04-

In [96]:
w1_udp.count(),w1_tcp.count()

(60, 60)

In [97]:
w1_udp = w1_udp.withColumnRenamed('window','window1').withColumnRenamed('proto','udp').withColumnRenamed('count','udp_count')
w1_tcp = w1_tcp.withColumnRenamed('window','window2').withColumnRenamed('proto','tcp').withColumnRenamed('count','tcp_count')

In [98]:
udp_tcp = w1_udp.join(w1_tcp,w1_udp.window1==w1_tcp.window2,how='inner')
udp_tcp.show(truncate=False)

+------------------------------------------+---+---------+------------------------------------------+---+---------+
|window1                                   |udp|udp_count|window2                                   |tcp|tcp_count|
+------------------------------------------+---+---------+------------------------------------------+---+---------+
|[2019-04-19 08:00:00, 2019-04-20 08:00:00]|17 |604      |[2019-04-19 08:00:00, 2019-04-20 08:00:00]|6  |462      |
|[2019-04-30 08:00:00, 2019-05-01 08:00:00]|17 |2586     |[2019-04-30 08:00:00, 2019-05-01 08:00:00]|6  |523      |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|17 |3574     |[2019-03-30 08:00:00, 2019-03-31 08:00:00]|6  |301      |
|[2019-04-07 08:00:00, 2019-04-08 08:00:00]|17 |5185     |[2019-04-07 08:00:00, 2019-04-08 08:00:00]|6  |505      |
|[2019-05-22 08:00:00, 2019-05-23 08:00:00]|17 |506      |[2019-05-22 08:00:00, 2019-05-23 08:00:00]|6  |387      |
|[2019-05-10 08:00:00, 2019-05-11 08:00:00]|17 |4566     |[2019-05-10 08

In [99]:
udp_tcp = udp_tcp.select('window1','udp','udp_count','tcp','tcp_count')

In [100]:
udp_tcp = udp_tcp.withColumnRenamed('window1','window').sort('window',ascending=True)
udp_tcp.show(truncate=False)

+------------------------------------------+---+---------+---+---------+
|window                                    |udp|udp_count|tcp|tcp_count|
+------------------------------------------+---+---------+---+---------+
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|17 |1908     |6  |730      |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|17 |4370     |6  |934      |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|17 |1219     |6  |529      |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|17 |401      |6  |392      |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|17 |8007     |6  |508      |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|17 |3574     |6  |301      |
|[2019-03-31 08:00:00, 2019-04-01 08:00:00]|17 |3437     |6  |500      |
|[2019-04-01 08:00:00, 2019-04-02 08:00:00]|17 |372      |6  |197      |
|[2019-04-02 08:00:00, 2019-04-03 08:00:00]|17 |1034     |6  |630      |
|[2019-04-03 08:00:00, 2019-04-04 08:00:00]|17 |9864     |6  |632      |
|[2019-04-04 08:00:00, 2019-04-05 08:00:00]|17 |271

In [101]:
udp_tcp_rate = udp_tcp.withColumn('udp_tcp_rate',round(udp_tcp.udp_count / udp_tcp.tcp_count,2))

In [102]:
udp_tcp_rate.sort('window',ascending=True).show(60,truncate=False)

+------------------------------------------+---+---------+---+---------+------------+
|window                                    |udp|udp_count|tcp|tcp_count|udp_tcp_rate|
+------------------------------------------+---+---------+---+---------+------------+
|[2019-03-25 08:00:00, 2019-03-26 08:00:00]|17 |1908     |6  |730      |2.61        |
|[2019-03-26 08:00:00, 2019-03-27 08:00:00]|17 |4370     |6  |934      |4.68        |
|[2019-03-27 08:00:00, 2019-03-28 08:00:00]|17 |1219     |6  |529      |2.3         |
|[2019-03-28 08:00:00, 2019-03-29 08:00:00]|17 |401      |6  |392      |1.02        |
|[2019-03-29 08:00:00, 2019-03-30 08:00:00]|17 |8007     |6  |508      |15.76       |
|[2019-03-30 08:00:00, 2019-03-31 08:00:00]|17 |3574     |6  |301      |11.87       |
|[2019-03-31 08:00:00, 2019-04-01 08:00:00]|17 |3437     |6  |500      |6.87        |
|[2019-04-01 08:00:00, 2019-04-02 08:00:00]|17 |372      |6  |197      |1.89        |
|[2019-04-02 08:00:00, 2019-04-03 08:00:00]|17 |1034  

In [103]:
udp_tcp_rate.select('udp_tcp_rate').describe().show()

+-------+------------------+
|summary|      udp_tcp_rate|
+-------+------------------+
|  count|                60|
|   mean| 8.190333333333331|
| stddev|15.937704677759601|
|    min|              1.02|
|    max|             116.6|
+-------+------------------+



In [104]:
ys.count()

265481

# mac => 442C051B66E4

In [105]:
ys.count()

265481

In [106]:
ys.show(3)

+-------------------+------------+-----+---------------+-----+
|                 ts|         mac|proto|            dip|dport|
+-------------------+------------+-----+---------------+-----+
|2019-03-25 15:01:27|EC3DFD2CC2A6|   17|183.136.184.150| 6003|
|2019-03-25 15:01:27|EC3DFD2CC2A6|   17|183.136.184.149| 6002|
|2019-03-25 15:01:27|EC3DFD2CC2A6|    6|183.136.184.149| 6002|
+-------------------+------------+-----+---------------+-----+
only showing top 3 rows



In [107]:
ys.groupBy('mac').count().sort('count',ascending=False).show()

+------------+------+
|         mac| count|
+------------+------+
|442C051B66E4|136676|
|BC1C8115ABCD| 94710|
|EC3DFD2CC2A6| 15390|
|4419B6A770D3| 13990|
|BC1C8104ED4B|  4685|
|E0B94D92067A|    30|
+------------+------+



In [22]:
ys.count()

316149

In [39]:
df = ys.filter(ys.mac =='442C051B66E4').select('ts','proto','bytes_out','pkts_out','dip','dport').distinct()

df_tmp = df.filter(df.proto != '1')
dd = df_tmp.groupBy(window('ts', '1 day'),'proto').count()
udp = dd.sort('window',ascending=True).filter(dd.proto == '17')
tcp = dd.sort('window',ascending=True).filter(dd.proto == '6')
udp = udp.withColumnRenamed('window','window1').withColumnRenamed('proto','udp').withColumnRenamed('count','udp_count')
tcp = tcp.withColumnRenamed('window','window2').withColumnRenamed('proto','tcp').withColumnRenamed('count','tcp_count')
udp_tcp = udp.join(tcp,udp.window1 == tcp.window2,how='inner')
udp_tcp = udp_tcp.select('window1','udp','udp_count','tcp','tcp_count')
udp_tcp = udp_tcp.withColumnRenamed('window1','window').sort('window',ascending=True)
udp_tcp_rate = udp_tcp.withColumn('udp_tcp_rate',round(udp_tcp.udp_count / udp_tcp.tcp_count,2))
#udp_tcp_rate.sort('window',ascending=False).show(60,truncate=False)


df_2 = df.groupBy(window('ts', '1 day'),'dport','bytes_out','pkts_out').count()
t1 = df_2.groupBy('window').agg({'dport':'count','bytes_out':'sum','pkts_out':'sum'})
t1 = t1.withColumnRenamed('window','window1')


final = udp_tcp_rate.join(t1,udp_tcp_rate.window == t1.window1,how = 'left')
final = final.select('window','udp', 'udp_count', 'tcp', 'tcp_count', 'udp_tcp_rate','count(dport)','sum(bytes_out)','sum(pkts_out)')
final.sort('window',ascending=False).show(60,truncate=False)

+------------------------------------------+---+---------+---+---------+------------+------------+--------------+-------------+
|window                                    |udp|udp_count|tcp|tcp_count|udp_tcp_rate|count(dport)|sum(bytes_out)|sum(pkts_out)|
+------------------------------------------+---+---------+---+---------+------------+------------+--------------+-------------+
|[2019-05-26 08:00:00, 2019-05-27 08:00:00]|17 |206      |6  |695      |0.3         |131         |89388.0       |811.0        |
|[2019-05-25 08:00:00, 2019-05-26 08:00:00]|17 |321      |6  |1056     |0.3         |118         |73649.0       |711.0        |
|[2019-05-24 08:00:00, 2019-05-25 08:00:00]|17 |313      |6  |1152     |0.27        |145         |98568.0       |896.0        |
|[2019-05-23 08:00:00, 2019-05-24 08:00:00]|17 |314      |6  |1143     |0.27        |124         |78959.0       |756.0        |
|[2019-05-22 08:00:00, 2019-05-23 08:00:00]|17 |314      |6  |1071     |0.29        |119         |74483.

In [40]:
final.agg(corr('udp_tcp_rate', 'count(dport)').alias('correlation_1')).show()

+------------------+
|     correlation_1|
+------------------+
|0.9942810967577108|
+------------------+



In [41]:
final.agg(corr('udp_tcp_rate', 'sum(bytes_out)').alias('correlation_2')).show()

+------------------+
|     correlation_2|
+------------------+
|0.9954024249844267|
+------------------+



In [42]:
final.agg(corr('udp_tcp_rate', 'sum(pkts_out)').alias('correlation_3')).show()

+------------------+
|     correlation_3|
+------------------+
|0.9953569129348593|
+------------------+



In [195]:
#output = final.repartition(1)
#output = output.toPandas()
#output = output.sort_values('window',ascending=False)
#output.to_excel('BC1C8104ED4B.xlsx',header=True,index=False)